In [16]:
import pandas as pd
import sqlite3

## PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  

In [17]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')

In [18]:
cur = con.cursor()

Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [19]:
myquery = """

SELECT subquery.name, SUM(subquery.totalcost) AS revenue
FROM
(SELECT f.name, 
CASE WHEN b.memid = 0 THEN b.slots*f.guestcost 
     ELSE b.slots*f.membercost END AS totalcost
FROM Bookings AS b
LEFT JOIN Members AS m
ON b.memid = m.memid
LEFT JOIN Facilities AS f
ON b.facid = f.facid) as subquery
GROUP BY subquery.name
HAVING revenue < 1000.0
ORDER BY revenue

"""

df = pd.read_sql_query(myquery, con)
df

,name,revenue
0,Table Tennis,180
1,Snooker Table,240
2,Pool Table,270


The three facilities with low revenue are table tennis, snooker table, and pool table.

Q11: Produce a report of members and who recommended them in alphabetic surname,firstname order

In [20]:
myquery = """

SELECT m1.memid, m1.firstname, m1.surname, m1.recommendedby, m2.firstname, m2.surname
FROM Members as m1
INNER JOIN Members as m2
ON m1.recommendedby = m2.memid
ORDER BY m1.surname, m1.firstname

"""

df = pd.read_sql_query(myquery, con)
df

,memid,firstname,surname,recommendedby,firstname,surname
0,15,Florence,Bader,9,Ponder,Stibbons
1,12,Anne,Baker,9,Ponder,Stibbons
2,16,Timothy,Baker,13,Jemima,Farrell
3,8,Tim,Boothe,3,Tim,Rownam
4,5,Gerald,Butters,1,Darren,Smith
5,22,Joan,Coplin,16,Timothy,Baker
6,36,Erica,Crumpet,2,Tracy,Smith
7,7,Nancy,Dare,4,Janice,Joplette
8,20,Matthew,Genting,5,Gerald,Butters
9,35,John,Hunt,30,Millicent,Purview


Q12: Find the facilities with their usage by member, but not guests

In [21]:
myquery = """

SELECT name, table_slots.total_slots
FROM Facilities as f
LEFT JOIN
(SELECT facid, SUM(slots) as total_slots
FROM Bookings
WHERE memid <> 0
GROUP BY facid) as table_slots
ON f.facid = table_slots.facid
ORDER BY name DESC

"""

df = pd.read_sql_query(myquery, con)
df

,name,total_slots
0,Tennis Court 2,882
1,Tennis Court 1,957
2,Table Tennis,794
3,Squash Court,418
4,Snooker Table,860
5,Pool Table,856
6,Massage Room 2,54
7,Massage Room 1,884
8,Badminton Court,1086


Q13: Find the facilities usage by month, but not guests 

In [22]:
myquery = """

SELECT name, table_slots.month, table_slots.year, table_slots.total_slots
FROM Facilities as f
LEFT JOIN
(SELECT facid, SUM(slots) as total_slots, substr(starttime,6,2) as month, substr(starttime,1,4) as year
FROM Bookings
WHERE memid <> 0
GROUP BY facid, month) as table_slots
ON f.facid = table_slots.facid
ORDER BY table_slots.month, table_slots.year, name DESC

"""

df = pd.read_sql_query(myquery, con)
df

,name,month,year,total_slots
0,Tennis Court 2,07,2012,123
1,Tennis Court 1,07,2012,201
2,Table Tennis,07,2012,98
3,Squash Court,07,2012,50
4,Snooker Table,07,2012,140
5,Pool Table,07,2012,110
6,Massage Room 2,07,2012,8
7,Massage Room 1,07,2012,166
8,Badminton Court,07,2012,165
9,Tennis Court 2,08,2012,345


In [23]:
cur.close()
con.close()